## 1 combine_data

In [1]:
import pandas as pd

In [2]:
train_path = 'C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\raw\\train.csv'
test_path = 'C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\raw\\test.csv'

In [3]:
train_df = pd.read_csv(train_path, index_col=0)
test_df = pd.read_csv(test_path, index_col=0)

In [4]:
train_df.shape, test_df.shape

((6261, 7), (2504, 6))

In [5]:
train_df["is_train"] = 1
test_df["is_train"] = 0

In [6]:
combined_df = pd.concat([train_df, test_df], sort=False)

In [7]:
combined_df.head(2)

,id,atm_group,address,address_rus,lat,long,target,is_train
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.01150,1
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.02971,1


In [8]:
combined_df.shape

(8765, 8)

In [9]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8765 entries, 0 to 2503
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8765 non-null   float64
 1   atm_group    8765 non-null   float64
 2   address      8765 non-null   object 
 3   address_rus  8345 non-null   object 
 4   lat          8345 non-null   float64
 5   long         8345 non-null   float64
 6   target       6261 non-null   float64
 7   is_train     8765 non-null   int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 616.3+ KB


In [10]:
combined_df.isnull().sum()

id                0
atm_group         0
address           0
address_rus     420
lat             420
long            420
target         2504
is_train          0
dtype: int64

## 2 find_gaps

In [11]:
combined_df[combined_df['address_rus'].isnull()]

,id,atm_group,address,address_rus,lat,long,target,is_train
40,4510.0,496.5,PER. BAZARNY 4 SPASSK-DALNY,NaN,NaN,NaN,0.039191,1
150,347.0,1022.0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,NaN,NaN,-0.145001,1
182,1591.0,1942.0,16A BEREJKOVSK NAB MOSCOW,NaN,NaN,NaN,0.080034,1
192,440.0,1942.0,113 MIRA AV. TOGLIATTI,NaN,NaN,NaN,-0.082956,1
214,556.0,1942.0,133A MOZHAYSKOE H/W. MOSCOW,NaN,NaN,NaN,0.061517,1
...,...,...,...,...,...,...,...,...
2346,3199.0,8083.0,"MOROZOVA P.L., 86 KHabarovsk",NaN,NaN,NaN,NaN,0
2399,5354.0,5478.0,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",NaN,NaN,NaN,NaN,0
2471,3333.0,8083.0,"OKTYABRSKAYA, 105 Tver",NaN,NaN,NaN,NaN,0
2474,2247.0,3185.5,"UL. TELEVIZORNAJA, KRASNOJARSK",NaN,NaN,NaN,NaN,0


## 3 data_enrichment_via_API

In [12]:
df = pd.DataFrame()

In [13]:
df

""


In [14]:
answer = {
    "source": "PER. BAZARNY 4            SPASSK-DALNY",
    "result": "Приморский край, г Спасск-Дальний, пер Базарный, д 4",
    "postal_code": "692245",
    "country": "Россия",
    "country_iso_code": "RU",
    "federal_district": "Дальневосточный",
    "region_fias_id": "43909681-d6e1-432d-b61f-ddac393cb5da",
    "region_kladr_id": "2500000000000",
    "region_iso_code": "RU-PRI",
    "region_with_type": "Приморский край",
    "region_type": "край",
    "region_type_full": "край",
    "region": "Приморский",
    "area_fias_id": None,
    "area_kladr_id": None,
    "area_with_type": None,
    "area_type": None,
    "area_type_full": None,
    "area": None,
    "city_fias_id": "e55554eb-c571-49d2-bf69-c3c4acf8b10b",
    "city_kladr_id": "2500001000000",
    "city_with_type": "г Спасск-Дальний",
    "city_type": "г",
    "city_type_full": "город",
    "city": "Спасск-Дальний",
    "city_area": None,
    "city_district_fias_id": None,
    "city_district_kladr_id": None,
    "city_district_with_type": None,
    "city_district_type": None,
    "city_district_type_full": None,
    "city_district": None,
    "settlement_fias_id": None,
    "settlement_kladr_id": None,
    "settlement_with_type": None,
    "settlement_type": None,
    "settlement_type_full": None,
    "settlement": None,
    "street_fias_id": "36781579-7de6-4a68-877b-bce878cd92b4",
    "street_kladr_id": "25000010000015600",
    "street_with_type": "пер Базарный",
    "street_type": "пер",
    "street_type_full": "переулок",
    "street": "Базарный",
    "house_fias_id": "5d57fcf1-29ba-4a45-95d9-ad43c4a74be4",
    "house_kladr_id": "2500001000001560001",
    "house_type": "д",
    "house_type_full": "дом",
    "house": "4",
    "block_type": None,
    "block_type_full": None,
    "block": None,
    "entrance": None,
    "floor": None,
    "flat_fias_id": None,
    "flat_type": None,
    "flat_type_full": None,
    "flat": None,
    "flat_area": None,
    "square_meter_price": None,
    "flat_price": None,
    "postal_box": None,
    "fias_id": "5d57fcf1-29ba-4a45-95d9-ad43c4a74be4",
    "fias_code": "25000010000000001560001",
    "fias_level": "8",
    "fias_actuality_state": "0",
    "kladr_id": "2500001000001560001",
    "capital_marker": "0",
    "okato": "05420000000",
    "oktmo": "05720000001",
    "tax_office": "2502",
    "tax_office_legal": "2502",
    "timezone": "UTC+10",
    "geo_lat": "44.5901175",
    "geo_lon": "132.8157288",
    "beltway_hit": None,
    "beltway_distance": None,
    "qc_geo": 4,
    "qc_complete": 9,
    "qc_house": 2,
    "qc": 1,
    "unparsed_parts": "ДАЛЬНЫ",
    "metro": None,
}

In [15]:
answer['result'], answer["geo_lat"], answer["geo_lon"], answer['qc_geo']

('Приморский край, г Спасск-Дальний, пер Базарный, д 4',
 '44.5901175',
 '132.8157288',
 4)

In [16]:
query1 = pd.DataFrame.from_dict([answer])

In [17]:
pd.concat([df, query1])

,source,result,postal_code,country,country_iso_code,federal_district,region_fias_id,region_kladr_id,region_iso_code,region_with_type,...,geo_lat,geo_lon,beltway_hit,beltway_distance,qc_geo,qc_complete,qc_house,qc,unparsed_parts,metro
0,PER. BAZARNY 4 SPASSK-DALNY,"Приморский край, г Спасск-Дальний, пер Базарны...",692245,Россия,RU,Дальневосточный,43909681-d6e1-432d-b61f-ddac393cb5da,2500000000000,RU-PRI,Приморский край,...,44.5901175,132.8157288,None,None,4,9,2,1,ДАЛЬНЫ,None


In [20]:
result = pd.read_csv('C:\\Users\\Юрий\\Documents\\MLOps\\atms_popularity\\data\\interim\\geocoded_data.csv')

In [23]:
result[['source', 'result', 'qc', 'qc_complete', 'qc_geo', 'qc_house']]

,source,result,qc,qc_complete,qc_geo,qc_house
0,PER. BAZARNY 4 SPASSK-DALNY,"Приморский край, г Спасск-Дальний, пер Базарны...",1,9,4,2
1,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,"Респ Татарстан, г Набережные Челны",1,3,4,10
2,16A BEREJKOVSK NAB MOSCOW,г Москва,1,3,4,10
